In [1]:
! pip install -q importlib rasterio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 41.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/sen2dwater/USANNIOMIT') # adjust to the correct one 
import importlib

In [4]:
from dataio.datahandler import datahandler
from dataio.datareader import datareader

from models.TDCNNv1 import TDCNNv1

from config import *
import matplotlib.pyplot as plt
import os

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
dataset_root_path = '/content/drive/MyDrive/SEN2DWATER' # adjust to the correct one
dh       = datahandler(dataset_root_path)
train_set, val_set = dh.split(SPLIT_FACTOR)

In [6]:
BATCH_SIZE = 70
SHAPE      = (64,64)
NORMALIZE  = True


NDVIval2 = datareader.generatorv2('NDVI',val_set,
                                        BATCH_SIZE,
                                        T_LEN,
                                        SHAPE,
                                        normalize=NORMALIZE)
NDWIval2 = datareader.generatorv2('NDWI',val_set,
                                        BATCH_SIZE,
                                        T_LEN,
                                        SHAPE,
                                        normalize=NORMALIZE)
NDDIval2 = datareader.generatorv2('NDDI',val_set,
                                        BATCH_SIZE,
                                        T_LEN,
                                        SHAPE,
                                        normalize=NORMALIZE)

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.models import load_model
import sklearn.preprocessing


# our trained ndwi model
# ndwi_model = load_model('/content/drive/MyDrive/results-lenovo/ndwi_model.h5')

# model from francesco
ndwi_model = load_model('/content/drive/MyDrive/sen2dwater/USANNIOMIT/ndwi_model2.h5')
# ndwi_model = load_model('/content/drive/MyDrive/results-lenovo2/ndwi_model2.h5')

ndvi_model = load_model('//content/drive/MyDrive/sen2dwater/USANNIOMIT/ndvi_model2.h5')
# ndvi_model = load_model('/content/drive/MyDrive/results-lenovo2/ndvi_model2.h5')

nddi_model = load_model('/content/drive/MyDrive/sen2dwater/USANNIOMIT/nddi_model2.h5')
# nddi_model = load_model('/content/drive/MyDrive/results-lenovo2/nddi_model2.h5')


In [ ]:
NDVI_input, NDVI_gt =  NDVIval2
NDWI_input, NDWI_gt =  NDWIval2
NDDI_input, NDDI_gt =  NDDIval2

In [ ]:
# # #print(NDDI_gt)
# # #print(NDVI_gt)
# # # 
# # # (50, 64, 64, 1)
# #print(NDDI_gt)
# import numpy as np
# for loc in NDDI_gt:
#   loc /= np.max(np.abs(NDDI_gt),axis=1)
# print(NDDI_gt)
# # max = 0
# # for loc in NDDI_gt:
# #   max = 0
# #   for time in loc:
# #       for pix in time:
# #         for jan in pix:
# #           if abs(jan) > max:
# #             max = jan
# #   loc /= max
# # print(NDDI_gt)

from tensorflow.keras.models import load_model
import numpy as np

print(NDDI_gt.shape)

# initial scaling function 
# def scaler(arr):
#   res = np.zeros_like(arr)
#   for i in range(arr.shape[0]):
#     for j in range(arr.shape[1]):
#         res[i][j] = sklearn.preprocessing.minmax_scale(arr[i][j], feature_range = (-1, 1))
#         # print(res.shape)
#   return res

# alternative
def scaler(arr):
  # use nanmin and nanmax
  # np.nanmax(img, axis=...)- max value for each row or col without considering nan
  # np.nandivide can divide with nans
  # substitute nan vals with 0 or some other smoothing value
  # arr[arr==np.nan] = 0 -> set nans to 0
  # scikit-image or scikit-learn to change nan pixels to nearest neighbours
  # in imshow: ax.imshow(img, vmin = real_min, vmax = real_max)

  # > correct the errors with nan by looking at scikit-learn and scikit-image
  # > loop through a subset of the dataset, save the max and min value of each image, and from this list of max and min get the real max and min, plug this in imshow
  # > first deactivate the normalize fn in dataloader
  # > get the max and min of each img using np.nanmax and np.nanmin, and then get the overall max and min for the whole dataset
  # > use this max and min to normalize images again, before training the model

  res = np.zeros_like(arr) 

  min_arr = np.nanmin(arr, axis = 0)
  max_arr = np.nanmax(arr, axis = 0)

  print(min_arr)
  print(max_arr) 
  for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        min_arr = np.min(arr[i][j])
        max_arr = np.max(arr[i][j])

        # print(min_arr)
        # print(max_arr)
        res[i][j] = (arr[i][j] - min_arr) / (max_arr - min_arr)
        # print(res.shape)
  return res

res = scaler(NDDI_gt)
# print(np.amin(NDDI_gt))
print(NDDI_gt.shape)


In [ ]:
# print(NDDI_input.shape, NDVI_gt.shape)
# print(res)
NDDI_gt = res

In [ ]:
ndvi_prediction = ndvi_model.predict(NDVI_input)
ndwi_prediction = ndwi_model.predict(NDWI_input)
nddi_prediction = nddi_model.predict(NDDI_input)

In [ ]:
shape = nddi_prediction.shape # Batch (), Width, Height, (NDWI, NNDI or NDVI)

In [ ]:

def getNDDI(NDVI,NDWI):
  return (NDVI-NDWI)/(NDVI+NDWI+0.0001)

nddi_from_NDVI_NDWI = []
for i in range(shape[0]): # get NDDI from NDWI and NDVI
  ndvi = ndvi_prediction[i]
  ndwi = ndwi_prediction[i]
  nddi_from_NDVI_NDWI.append(getNDDI(ndvi,ndwi))

NDDItheoreticGT = getNDDI(NDVI_gt,NDWI_gt)

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = BATCH_SIZE, figsize = (BATCH_SIZE*5,3*5))
for i in range(BATCH_SIZE):
  axes[0,i].imshow(ndvi_prediction[i,:,:,0])
  axes[1,i].imshow(NDVI_gt[i,:,:,0])

  axes[0,i].set_title('NDVI Prediction')
  axes[1,i].set_title('NDVI Ground Truth')

  axes[0,i].axis(False)
  axes[1,i].axis(False)

fig.tight_layout()
plt.show()
plt.close()

fig, axes = plt.subplots(nrows = 2, ncols = BATCH_SIZE, figsize = (BATCH_SIZE*5,3*5))
for i in range(BATCH_SIZE):
  axes[0,i].imshow(ndwi_prediction[i,:,:,0])
  axes[1,i].imshow(NDWI_gt[i,:,:,0])

  axes[0,i].set_title('NDWI Prediction')
  axes[1,i].set_title('NDWI Ground Truth')

  axes[0,i].axis(False)
  axes[1,i].axis(False)

fig.tight_layout()
plt.show()
plt.close()

fig, axes = plt.subplots(nrows = 3, ncols = BATCH_SIZE, figsize = (BATCH_SIZE*5,3*5))
for i in range(BATCH_SIZE):
  axes[0,i].imshow(nddi_prediction[i,:,:,0])
  axes[1,i].imshow(NDDI_gt[i,:,:,0])
  axes[2,i].imshow(NDDItheoreticGT[i,:,:,0])

  axes[0,i].set_title('NDDI Prediction')
  axes[1,i].set_title('NDDI Ground Truth')
  axes[2,i].set_title('NDDI Theoretic GT')

  axes[0,i].axis(False)
  axes[1,i].axis(False)
  axes[2,i].axis(False)

fig.tight_layout()
plt.show()
plt.close()

fig, axes = plt.subplots(nrows = 2, ncols = BATCH_SIZE, figsize = (BATCH_SIZE*5,3*5))
for i in range(BATCH_SIZE):
  axes[0,i].imshow(nddi_from_NDVI_NDWI[i,:,:,0])
  axes[1,i].imshow(NDDI_gt[i,:,:,0])

  axes[0,i].set_title('NDDI Prediction from NDVI and NDWI')
  axes[1,i].set_title('NDDI Ground Truth')

  axes[0,i].axis(False)
  axes[1,i].axis(False)

fig.tight_layout()
plt.show()
plt.close()